In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.restaurants_db
collection = db.items

In [4]:
# URL of page to be scraped
url = 'https://www.findmeglutenfree.com/search?lat=33.4483771&lng=-112.07403729999999&q=&a=phoenix%2C+az&local=t'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [5]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
  <meta content="Local gluten free places near phoenix, az" name="description"/>
  <title>
   Local Gluten Free Places phoenix, az
  </title>
  <link crossorigin="anonymous" href="https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/css/bootstrap.min.css" integrity="sha384-MCw98/SFnGE8fJT3GXwEOngsV7Zt27NXFoaoApmYm81iuXoPkFOJwJ8ERdknLPMO" rel="stylesheet"/>
  <link crossorigin="anonymous" href="https://use.fontawesome.com/releases/v5.3.1/css/all.css" integrity="sha384-mzrmE5qonljUremFsqc01SB46JvROS7bZs3IO2EmfFsd15uHvIt+Y8vEf7N7fWAU" rel="stylesheet"/>
  <link href="/main/css/style_001.css" rel="stylesheet"/>
  <link href="https://www.findmeglutenfree.com/img/image_src.png" rel="image_src"/>
  <meta content="app-id=431006818" name="apple-itunes-app"/>
  <meta content="Local Gluten Free Places phoenix, az" property="og:title"/>
  <meta 

In [8]:
# Examine the results, then determine element that contains sought info
# results are returned as an iterable list
results = soup.find_all('div', class_='sl-title')
print(results)

[<div class="sl-title"><h2><span class="mr-2 sl-ad align-middle">AD</span><a class="align-middle" href="/biz/picazzos-healthy-italian-kitchen/2291004?sp=t">Picazzo's Healthy Italian Kitchen</a></h2></div>, <div class="sl-title"><h2><a class="align-middle" href="/biz/the-counter/6679924027162624">The Counter</a></h2></div>, <div class="sl-title"><h2><a class="align-middle" href="/biz/cibo/13365163">Cibo</a></h2></div>, <div class="sl-title"><h2><a class="align-middle" href="/biz/the-strand/6013929465839616">The Strand</a></h2></div>, <div class="sl-title"><h2><a class="align-middle" href="/biz/grabbagreen/5093265368416256">Grabbagreen</a></h2></div>, <div class="sl-title"><h2><a class="align-middle" href="/biz/squid-ink-sushi-bar/5644230971097088">Squid Ink Sushi Bar</a></h2></div>, <div class="sl-title"><h2><a class="align-middle" href="/biz/matts-big-breakfast/6249385940221952">Matt's Big Breakfast</a></h2></div>, <div class="sl-title"><h2><a class="align-middle" href="/biz/chase-fiel

In [5]:
# Loop through returned results
for result in results:
    # Error handling
    try:
        # Identify and return title of listing
        title = result.find('div', class_='sl-title').text
        # Identify and return price of listing
        city = result.a.span.text
        # Identify and return link to listing
        link = result.a['href']

        # Run only if title, price, and link are available
        if (title and price and link):
            # Print results
            print('-------------')
            print(title)
            print(price)
            print(link)

            # Dictionary to be inserted as a MongoDB document
            post = {
                'title': title,
                'price': price,
                'url': link
            }

            collection.insert_one(post)

    except Exception as e:
        print(e)

-------------
Martin 12 string guitar
$2100
https://newjersey.craigslist.org/msg/d/morris-plains-martin-12-string-guitar/6953889446.html
-------------
GUITAR WORLD MAGAZINES...140 FOR $80...NICE
$80
https://newjersey.craigslist.org/bks/d/garwood-guitar-world-magazines140-for/6948559434.html
-------------
Zoom 606 Guitar Effects
$48
https://newjersey.craigslist.org/msg/d/pine-brook-zoom-606-guitar-effects/6953804170.html
-------------
WASHBURN BASS GUITAR
$125
https://newjersey.craigslist.org/msg/d/newton-washburn-bass-guitar/6942209189.html
-------------
Vintage Ovation Matrix 1132-1, Acoustic/Electric Guitar
$400
https://newjersey.craigslist.org/msg/d/succasunna-vintage-ovation-matrix/6953741774.html
-------------
On Stage 5 Guitar Stand like new
$40
https://newjersey.craigslist.org/msg/d/highland-lakes-on-stage-5-guitar-stand/6948626382.html
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
-------------
Electric guitar gig bag
$15
https://newjersey.

In [6]:
# Display items in MongoDB collection
listings = db.items.find()

for listing in listings:
    print(listing)

{'_id': ObjectId('5d4f24372b1bdef7fd192282'), 'title': 'Martin 12 string guitar', 'price': '$2100', 'url': 'https://newjersey.craigslist.org/msg/d/morris-plains-martin-12-string-guitar/6953889446.html'}
{'_id': ObjectId('5d4f24392b1bdef7fd192283'), 'title': 'GUITAR WORLD MAGAZINES...140 FOR $80...NICE', 'price': '$80', 'url': 'https://newjersey.craigslist.org/bks/d/garwood-guitar-world-magazines140-for/6948559434.html'}
{'_id': ObjectId('5d4f24392b1bdef7fd192284'), 'title': 'Zoom 606 Guitar Effects', 'price': '$48', 'url': 'https://newjersey.craigslist.org/msg/d/pine-brook-zoom-606-guitar-effects/6953804170.html'}
{'_id': ObjectId('5d4f24392b1bdef7fd192285'), 'title': 'WASHBURN BASS GUITAR', 'price': '$125', 'url': 'https://newjersey.craigslist.org/msg/d/newton-washburn-bass-guitar/6942209189.html'}
{'_id': ObjectId('5d4f24392b1bdef7fd192286'), 'title': 'Vintage Ovation Matrix 1132-1, Acoustic/Electric Guitar', 'price': '$400', 'url': 'https://newjersey.craigslist.org/msg/d/succasunna-